## Best of Generation Accuracy Simulation  (Mutation Rate set to 0.025) 

In [1]:
import time 
import random
import numpy as np
import itertools
import networkx as nx
from statistics import mean
import matplotlib.pyplot as plt
import heapq

In [2]:
# ***
# Generate Log-Normal Distributed Graph 
# Generate coordinates for n nodes 
# n: Number of nodes 
# scale: Scaling factor for coordinates generated between [0, 1)
def generate_nodes(n, scale):
    return np.random.rand(n, 2) * scale

# Calculate the straight line distance between coordinates  
#     by applying Pythagoras' theorem (a^2 + b^2 = c^2)
# a: First coordinate pair 
# b: Second coordinate pair 
def pythag(a, b):
    return np.sqrt(np.square(a[0] - b[0]) + np.square(a[1] - b[1]))

def depth_factor(i, j):
    return np.exp(-pythag(i, j))

def surface_factor(si, sj):
    return (si + sj)

def nx_log_normal(size, degree):
    # Generate coordinates in range [0, 1]
    scale = 1
    nodes = generate_nodes(size, scale)
    
    # Define surface weights for each node 
    surfaces = np.random.lognormal(mean = 0.0, sigma = 1.0, size = size)
    
    # Define the surface factor between every node pair 
    # Larger surfaces have greater probability to connect 
    sf = [[surface_factor(i, j) for i in surfaces] for j in surfaces] 
    
    # Define the depth factor between every pair of nodes 
    # Geometrically closer nodes have higher depth factor 
    df = [[depth_factor(i, j) for i in nodes] for j in nodes]
    
    # Calculate the existence probability for each edge 
    # depth_factor_ij * surface_factor_ij 
    ep = np.multiply(df, sf)
    total_ep = np.sum(ep)
    
    # Roulette wheel 
    # Create an edge (i, j) with Pr. proportional to ep_ij 
    edges_set = []
    
    while degree > 0:
        pr_sum = 0.0 
        ptr = random.random()
        
        for i, j in itertools.product(range(size), range(size)):
                if i == j:
                    continue 
                
                pr = ep[i][j] / total_ep
                pr_sum += pr 
                
                if pr_sum > ptr: 
                    edges_set.append((i, j, {'weight': pr}))
                    break
        
        degree = degree - 1 
    
    G = nx.Graph()
    G.add_nodes_from(range(size))
    G.add_edges_from(edges_set)
    
    return G 

# Get a connected log normal 
def connected_log_normal(size, degree):
    # Set a hard limit on the number of models generated
    # before exit
    max_iterations = 100
    
    # Generate models until connected
    for i in range(max_iterations):
        G = nx_log_normal(size, degree)
        
        # Check if G is connected 
        connected = True
        for node in range(size):
            if G.degree[node] == 0:
                connected = False
    
        if connected:
            return G
    
    # Return the false flag and null model
    return None 


# ***
# Generate Barabasi-Albert Model 
def barabasi_albert_node(G, n):
    # Node & Degree for each node in G excluding new node 
    degree = [d for (n, d) in G.degree]
    # Probability for each node 
    # There are no edges from node n 
    pr = np.divide(degree, (2 * len(G.edges())))
    
    return np.random.choice(G.nodes(), p = pr)


def barabasi_albert_edge(G, n): 
    dest = barabasi_albert_node(G, n)
    edge = (n, dest)
    
    # Recurse to add new non-existing edge 
    if edge in G.edges():
        barabasi_albert_edge(G, n)
        return
    
    # Avoid self-edges
    # Self edges should not occur (probability to choose n is zero)
    # However, they do appear in this implementation FIX LATER
    if n == dest:
        barabasi_albert_edge(G, n)
        return
    
    G.add_edge(n, dest)
    
# Produce a scale-free network represented as a networkx graph 
#    object with initial size m and final size n
# m: Initial size of graph
# n: Final size of graph 
# return: Networkx graph model 
def barabasi_albert(m, n):
    # Begin with a fully connected graph size m
    G = nx.complete_graph(m)
    
    for n in range(m, n):
        G.add_node(n)
        
        # Connect new node to m existing nodes 
        for i in range(m):
            barabasi_albert_edge(G, n)
    
    return G 


# ***
# Generate a Watts-Strogatz Networkx graph 
# Return every adjacent pair / edge in the ring lattice 
#       structure 
# Enumerate the edges in a ring lattice of degree k 
def ring_adjacents(nodes, k):
    hk = int(np.floor(k / 2))
    
    edges = []
    for node in nodes:
        # Identify each adjacent node 
        for j in range((node + 1), (node + hk + 1)):
            v = nodes[j % len(nodes)]  # Modulo enforces loop back to start node  
            edges.append((node, v))
    
    return edges

def ring_network(n, k):
    nodes = range(n)
    edges = ring_adjacents(nodes, k)
    
    # Create networkx graph 
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    
    return G

def ws_rewire(G, p):
    for i, j in G.edges():
        if random.random() < p:
            G.remove_edge(i, j)
            nodes = [n for n in list(G.nodes) if n != i  and n not in G[i]]
            # Create new edge
            u, v = i, random.choice(nodes)
            G.add_edge(u, v)
            
# Generate a networkx graph with small world properties (Watts-Strogatz)
def watts_strogatz(n, k, p):
    # Generate the edge list of a ring lattice 
    #     of size n and degree k 
    ring = ring_network(n, k)
    
    # Randomly reassign edges with probability p
    ws_rewire(ring, p)
    
    return ring


# Randomly assign edge weights in range to graph
def assign_random_weights(G, wmin, wmax, w = None):
    if w is None:
        for edge in G.edges():
            G[edge[0]][edge[1]]['weight'] = np.random.uniform(low=wmin, high=wmax)
    else:
        for edge in G.edges():
            G[edge[0]][edge[1]]['weight'] = w
            
            
def weighted_watts_strogatz(n, k, p):
    g = watts_strogatz(n, k, p)
    assign_random_weights(g, 0, 100)
    return g 


def weighted_log_normal(size, degree):
    g = connected_log_normal(size, degree)
    assign_random_weights(g, 0, 100)
    return g

def weighted_barabasi_albert(m, n):
    g = barabasi_albert(m, n)
    assign_random_weights(g, 0, 100)
    return g

In [3]:
class GA:
    # CLASS WIDE PROPERTIES 
    # Reproduction rate (half of population used as parents)
    R_RATE = 0.5 
    
    # ***
    # Constructor 
    def __init__(self, ITERATIONS = 30, THRESHOLD = 12, 
                    P_SIZE = 100, M_RATE = 0.025, RI = 0.1, 
                EI = 0.1, IM = 0.8):
        self.ITERATIONS = ITERATIONS
        self.THRESHOLD = THRESHOLD 
        self.P_SIZE = P_SIZE
        self.M_RATE = M_RATE
        
        # Flag which immigrant scheme to use 
        self.flag_riga = False
        self.flag_eiga = False
        
        # Immigrant scheme variables 
        self.RI = RI   # Proportion of population replaced with Random Immigrants
        self.EI = EI   # Proportion of population replaced with Elitist Immigrants 
        self.IM = IM   # Probability to mutate elitist immigrant 
        
        # Graph
        self.G = None 
        self.src = 0 
        self.dst = 0
        
        # Generation Variables 
        self.reset_generation() 
    
    
    # ***
    # Setter methods for RIGA/EIGA/HIGA configuration 
    def set_riga(self):
        self.flag_riga, self.flag_eiga = True, False 
    
    def set_eiga(self):
        self.flag_riga, self.flag_eiga = False, True 
        
    def set_higa(self):
        self.flag_eiga, self.flag_riga = True, True 
    
    def set_standard(self):
        self.flag_eiga, self.flag_riga = False, False 
    
    def set_mode(self, mode):
        if mode == "eiga":
            self.set_eiga()
        elif mode == "riga":
            self.set_riga()
        elif mode == "higa":
            self.set_higa()
        elif mode == "standard":
            self.set_standard()
        else:
            return 
    
    
    # ***
    def path_length(self, p):
        return sum(self.G[i][j]['weight'] for i, j in zip(p, p[1:]))
    
    # ***
    # Return fitness given path and graph 
    def fit(self, p):
        # Zero fitness for invalid path 
        if not nx.is_path(self.G, p):
            return 0

        # Fitness for valid path 
        return self.path_length(p) ** (-1)
    
    
    # ***
    # Function to define a single new random path from node s to node r
    # in a network supplied as a Networkx graph 
    # G: Graph / network as networkx
    # s: Start node 
    # r: Destination node 
    def gen_path(self, s, r):
        path = [s]
        
        current = s
        while current != r: 
            # Neighbours exluding nodes in path 
            neighbours = [n for n in self.G[current] if n not in path]

            # Sometimes, neighbours is empty 
            # In which case, retry 
            if not neighbours:
                return self.gen_path(s, r)

            # Select neighbour node with uniform probability  
            nxt = np.random.choice(neighbours) 

            # List and visit next node 
            path.append(nxt)
            current = nxt 

        return path
    
    
    # ***
    # Function to generate a population of n random paths s->r
    # G: Graph as networkx
    # s: Start node
    # r: Destination node
    # n: Number of paths / size of population 
    def gen_population(self, size):
        population = []
        for i in range(size):
            # Generate a new candidate path
            candidate = self.gen_path(self.src, self.dst)
            # Generate a new candidate 
            population.append(candidate)
        
        return population 
    
    
    # ***
    # Perform crossover between two paths s->r
    # The two paths must have at least one common element 
    # From the set of common elements, one is selected, v 
    # The paths are redifined as s -> v, v -> r  
    # The path segments are recombined to create two children
    # a: The first path
    # b: The second path 
    def crossover(self, a, b):
        common = [] 
        for ind_a, i in enumerate(a):
            for ind_b, j in enumerate(b):
                if i == j:
                    common.append((ind_a, ind_b))

        # Randomly select a one common element to exchange
        target = np.random.choice(len(common))

        # Retrieve indexes
        ind_a, ind_b = common[target]

        # Create new arrays
        chd_a = a[:ind_a] + b[ind_b:]
        chd_b = b[:ind_b] + a[ind_a:]

        return chd_a, chd_b
    
    
    # ***
    # Updated mutation function 
    # Deal with possibility that elements of path no longer exist, 
    # except for src and dst 
    def mutate(self, path):
        indexes = np.random.choice((len(path) - 1), size = (len(path) - 1), replace = False)
        
        for i in indexes:
            if self.G.has_node(path[i]):
                rnd_path = self.gen_path(path[i], self.dst)
                return path[:i] + rnd_path
        
        # No change if no nodes in path to mutate 
        return path
    
    
    # ***
    # Compute the fitness value of each hypotheses 
    # p: Population
    # g: Graph 
    # Return: fitness list, maximum fitness value, sum of fitness values 
    def evaluate(self, p):
        # Evaluate the fitness of the population 
        fitness = [self.fit(h) for h in p]

        return fitness, max(fitness), sum(fitness), mean(fitness)
    
    
    # ***
    # Stochastic universal selection 
    #     - Uses a single random value to sample all solutions 
    # n: Number of hypotheses to select
    # population: Population of hypotheses
    # fitness: Fitness array
    # ftotal: Sum of fitness values 
    # target: Target array to append selected hypotheses 
    def stochastic_universal(self, n, target):
        step = self.ftotal / n                       # Size of selection bins 
        start = random.uniform(0, step)     # Random in range [0, step)

        # Each pointer value indicates a selection threshold
        # When the sum of probabilities exceeds a pointer, the current hypotheses is selected 
        pointers = [(start + (i * step)) for i in range(0, n)]

        for ptr in pointers:
            ind = 0               # Current index 
            running_sum = 0.0       # Sum of probabilities (fit(h) / total_fitness)

            # Until the threshold is exceeded 
            while running_sum < ptr: 
                running_sum += self.fitness[ind]
                ind += 1 

            target.append(self.population[ind - 1])
    
    
    # ***
    # Set all generation parameters to zero/empty/none before
    # initialisation and beginning a new run 
    def reset_generation(self):
        self.start, self.end = 0, 0
        
        # GA Variables 
        self.elite = []
        self.fitness = []
        self.population = [] 
        
        # Generation stats
        self.generation_count = 0 
        self.num_stable_generations = 0 
        self.fmax, self.favg, self.ftotal = 0, 0, 0

        self.history = {"max": [], "avg": []}
    
    
    # ***
    # Function to initialise the population 
    # with starting graph G, src and destination 
    def initialise(self, G, src, dst):
        self.G = G.copy() 
        self.src = src
        self.dst = dst
        
        # Reset stats
        self.reset_generation()
        
        # Generate a random population 
        # Graph; start node; dest node; size of population 
        self.population = self.gen_population(self.P_SIZE) 
        self.fitness, self.fmax, self.ftotal, self.favg = self.evaluate(self.population)
        
        # Initialise previous generation elite at P(0) 
        self.elite = self.population[np.argmax(self.fitness)]
    
    
    # ***
    # Return the best candidate in current population 
    def get_best_candidate(self): 
        if (not self.fitness) or (not self.population):
            return [] 
        
        ind = np.argmax(self.fitness)
        best = self.population[ind]
        return (self.path_length(best), 
                self.fitness[ind], best)
        
        
    # ***
    # Get performance results from current generation 
    def get_results(self):
        # Return stats and performance 
        length, fit, best = self.get_best_candidate()
        runtime = self.end - self.start 
        
        results = {"generations": self.generation_count, 
                   "runtime": runtime,
                   "history": self.history, 
                   "solution": best, 
                   "length": length, 
                   "fitness": fit}
        
        return results 
    
    
    # *** 
    # Generate elitist immigrants from P(t-1)
    def gen_ei(self, gen):
        ei_list = []
        n_ei = int(self.EI * self.P_SIZE)

        # Generate ei*p elitist immigrants 
        for i in range(n_ei):
            # Mutate elite from p(t - 1)
            if random.random() < self.IM:
                ei_list.append(self.mutate(self.elite)) 
            else:
                ei_list.append(self.elite)

        return ei_list 
    
    
    # ***
    # Generate random immigrants
    def gen_ri(self, gen):
        n_ri = int(self.RI * self.P_SIZE)
        return self.gen_population(n_ri)
    
    
    # ***
    # Run a single generation of GA 
    def run_generation(self):
         # Append to training history 
        self.history["max"].append(self.fmax)
        self.history["avg"].append(self.favg)
        
        # Create new generation 
        gen = [] 
        
        # Select ((1 - r) * p) hypotheses to persist into the next generation
        # Hypotheses are selected with fitness proportional probability 
        # Pr(hi) = fit(hi) / sumj(fit(hj))
        num_hypotheses = int((1 - self.R_RATE) * self.P_SIZE)
        self.stochastic_universal(num_hypotheses, gen)
        
        # Select (r * p) / 2 pairs of parents for crossover 
        # Parent hypotheses are chosen with fitness proportional probability 
        # Pr(hi) = fit(hi) / sumj(fit(hj))
        parents = [] 
        num_parents = int(self.R_RATE * self.P_SIZE) 
        self.stochastic_universal(num_parents, parents)
        
        # Crossover pairs of parents 
        pairs = zip(parents, parents[1:])  # Zip into pairs [1, 2, 3] -> (1, 2), (2, 3)
        for a, b in pairs:
            e, f = self.crossover(a, b)   # Crossover parents a,b to get children c,d 
            gen.append(e) 
            gen.append(f)
            
        # Choose m percent of new population uniformly to mutate 
        m_size = int(self.M_RATE * self.P_SIZE)
        for i in np.random.choice(len(gen), m_size):
            gen[i] = self.mutate(gen[i]) 
            
        # *** BEGIN HIGA SCHEME ***
        
        # Evaluate interim population 'gen'
        (gf, gmax, gtotal, gavg) = self.evaluate(gen)
        
        immigrants = [] 
        
        # If EIGA is true 
        if self.flag_eiga:
            # Check that elite immigrant is still valid E(t - 1)
            # If not, then use current best as new elite 
            if not nx.is_path(self.G, self.elite):
                # Re-evaluate current population to get new elite
                temp_fitness = self.evaluate(self.population)[0]
                # Use current best instead 
                ind = np.argmax(temp_fitness)
                self.elite = self.population[ind]
            
            immigrants = immigrants + self.gen_ei(immigrants)
        
        if self.flag_riga:
            immigrants = immigrants + self.gen_ri(immigrants)
    
        # Replace the ri*p worst candidates with immigrants 
        # O(n log k) to find k smallest elements in array of n 
        #   - Use min heap of size k, compare each element to heap min, i
        #        if smaller then remove max and replace 
        n_immigrants = len(immigrants)
        if n_immigrants:
            heap = []
            for i in range(n_immigrants):
                # Tuple (fitness, index) and order heap on first element
                heapq.heappush(heap, (-(gf[i]), i)) 
            heapq.heapify(heap)

            # Add smaller elements and maintian heap 
            for i in range(n_immigrants, len(gen)):
                if -(gf[i]) < heap[0][0]:
                    heap[0] = heap[-1]
                    heap.pop()
                    heap.append(((-gf[i]), i))
                    heapq.heapify(heap)

            # Replace worst candidates 
            for i in range(n_immigrants):
                ind = heap[i][1]
                gen[ind] = immigrants[i]
    
            # Set elite 
            # Elite is best of P(t - 1), 
            # Each generation G(t) works on P(t) to Generate P(t + 1)
            self.elite = self.population[np.argmax(self.fitness)]
    
        # *** END of HI SCHEME *** 
        
        # Population equal to copy of generation 
        self.population = list(gen) 
        
        # Evaluate the new population fitness
        _fmax = self.fmax
        self.fitness, self.fmax, self.ftotal, self.favg = self.evaluate(self.population)
        
        # Check if maximum fitness solution has changed 
        if _fmax == self.fmax:
            self.num_stable_generations += 1 
        else:
            # Reset when max fitness changes 
            self.num_stable_generations = 0 
        
        # Increment generation count
        self.generation_count += 1 
        
    
    # ***
    def get_generation_stats(self):
        return (self.fitness, self.fmax, self.favg)
        
        
    # ***
    # Run full GA to ITERATIONS or THRESHOLD 
    def run(self):
        self.start = time.time()
        while (self.num_stable_generations < self.THRESHOLD) and (self.generation_count < self.ITERATIONS):
            self.run_generation()
        self.end = time.time()
    
    
    # ***
    def set_iterations(self, itr):
        self.ITERATIONS = itr 
    
    
    # ***
    def set_threshold(self, thr):
        self.THRESHOLD = thr
        

    # ***
    def set_graph(self, graph):
        self.G = graph.copy() 
        
        
    def get_history(self ):
        return self.history

For a graph of 500 nodes of each type, run each GA variation standard, riga, higa, eiga ten times. 
Take the fitness history of each run.
standard 10x100
riga 10x100... etc
Also, for each run, record the true shortest path. 
10 x 100 shorest path list 
Then, divide each generation history by the corresponding shortest path list element wise to get a new set of values representing the accuracy (value of 1 indicates no error). 
Take the average of each accuracy list. Take the average of the averages. 
That gives a single value indicating the average accuracy for the GA type on that graph, with that node set.
The outcome should be a table of Ga type vs Graph type (Topology, dynamic with size 500 and Pr 0.05)

In [4]:
# With Probability Pr randomly rewire each edge to a new destination node
# Destination node must not be self, or already connected 
# G: Networkx Graph
# pr: Probability to rewire 
def dynamic_step_EDGE(G, pr = 0.0):
    # For all edges 
    for i, j in G.edges():
        # With probability Pr 
        if random.random() < pr:            
            weight = G[i][j]['weight']
            
            if G.degree[j] == 1:
                continue 
                
            # Rewire the edge to from j to v
            G.remove_edge(i, j)
            nodes = [n for n in list(G.nodes) if n != i  and n not in G[i]]
            # Create new edge
            u, v = i, random.choice(nodes)
            G.add_edge(u, v, weight=weight)


# ***
def dynamic_step_WEIGHT(G, Pr, minw = 1, maxw = 100):
    # For each edge and weight
    for i, j, w in G.edges(data = True):
        if random.random() < Pr:
            G[i][j]['weight'] = np.random.uniform(minw, maxw)

        
# ***
# note: the dynamic node step removes nodes from the graph 
# For this reason, a copy of the parametised graph is taken
# nodes are removed from the copy, the copy is returned 
# If the original graph is directly effected, all nodes will
# be removed in a few iterations, leading to failure of simulations. 
def dynamic_step_NODE(G, Pr, avoid, minw = 1, maxw = 100):
    # Take copy of input graph 
    G_ = G.copy() 
    
    removed = []
    reconnect = []
    # Randomly remove nodes
    for node in list(G_.nodes()):
        if node in avoid:
            continue 
        if random.random() < Pr:
            # Check if any nodes need to be reconnected
            for j in G_[node]:
                if G_.degree[j] == 1:
                    reconnect.append(j) 
            
            G_.remove_node(node)
            removed.append(node)
    
    # Reconnect lonely, disconnected nodes 
    for j in reconnect:
        if j in G_.nodes():
            i = np.random.choice(G_.nodes())
            G_.add_edge(j, i, weight = random.uniform(minw, maxw))
    
    return G_

In [5]:
def experiment_node(graph_type, trials, generations):
    Pr = 0.05 # Probability to effect a change 
    n, k, p = 500, 4, 0.25
    m = 3 # BA 
    degree = 2500 # LN 
    
    stnd_ga = GA() 
    riga_ga = GA()
    eiga_ga = GA()
    higa_ga = GA()
    
    # Run trials 
    stnd_results = []
    riga_results = []
    eiga_results = []
    higa_results = [] 
    sp_set = [] 
    
    for trial in range(trials):
        # Generate a new graph 
        if graph_type == "WS":
            G = weighted_watts_strogatz(n, k, p)
        elif graph_type == "BA":
            G = weighted_barabasi_albert(m, n)
        else:
            G = weighted_log_normal(n, degree)
        
        src, dst = np.random.choice(n, size = 2, replace = False)
        # Initialise GA  
        stnd_ga.initialise(G, src, dst)
        riga_ga.initialise(G, src, dst)
        eiga_ga.initialise(G, src, dst)
        higa_ga.initialise(G, src, dst)
        
        # Run generations 
        sp = []
        for generation in range(generations):            
            # Find true shortest path 
            path = nx.dijkstra_path(stnd_ga.G, src, dst, weight='weight')
            length = path_length(stnd_ga.G, path)
            sp.append(length ** (-1))
    
            # Run generation
            stnd_ga.run_generation()
            riga_ga.run_generation()
            eiga_ga.run_generation()
            higa_ga.run_generation() 
        
            # Update graph (avoid src, dst)
            G_temp = dynamic_step_NODE(G, Pr, avoid=(src, dst))
            stnd_ga.set_graph(G_temp)
            riga_ga.set_graph(G_temp)
            eiga_ga.set_graph(G_temp)
            higa_ga.set_graph(G_temp)
            
        # Get the results from the run  
        sp_set.append(sp)
        stnd_results.append(stnd_ga.get_history()["max"])
        riga_results.append(riga_ga.get_history()["max"])
        eiga_results.append(eiga_ga.get_history()["max"])
        higa_results.append(higa_ga.get_history()["max"])
    
    return stnd_results, riga_results, eiga_results, higa_results, sp_set

In [6]:
def experiment_weight(graph_type, trials, generations):
    Pr = 0.05 # Probability to effect a change 
    n, k, p = 500, 4, 0.25
    m = 3 # BA 
    degree = 2500 # LN 
    
    stnd_ga = GA() 
    riga_ga = GA()
    eiga_ga = GA()
    higa_ga = GA()
    
    # Run trials 
    stnd_results = []
    riga_results = []
    eiga_results = []
    higa_results = [] 
    sp_set = [] 
    
    for trial in range(trials):
        # Generate a new graph 
        if graph_type == "WS":
            G = weighted_watts_strogatz(n, k, p)
        elif graph_type == "BA":
            G = weighted_barabasi_albert(m, n)
        else:
            G = weighted_log_normal(n, degree)
            
        G_ = G.copy() # Copy to maintain original state G 
        src, dst = np.random.choice(n, size = 2, replace = False)
        # Initialise GA
        stnd_ga.initialise(G, src, dst)
        riga_ga.initialise(G, src, dst)
        eiga_ga.initialise(G, src, dst)
        higa_ga.initialise(G, src, dst)
        
        # Run generations 
        sp = []
        for generation in range(generations):            
            # Find true shortest path 
            path = nx.dijkstra_path(stnd_ga.G, src, dst, weight='weight')
            length = path_length(stnd_ga.G, path)
            sp.append(length ** (-1))
    
            # Run generation
            stnd_ga.run_generation()
            riga_ga.run_generation()
            eiga_ga.run_generation()
            higa_ga.run_generation() 
        
            # Update graph (avoid src, dst)
            dynamic_step_WEIGHT(G_, Pr)
            
            stnd_ga.set_graph(G_)
            riga_ga.set_graph(G_)
            eiga_ga.set_graph(G_)
            higa_ga.set_graph(G_)
            
        # Get the results from the run  
        sp_set.append(sp)
        stnd_results.append(stnd_ga.get_history()["max"])
        riga_results.append(riga_ga.get_history()["max"])
        eiga_results.append(eiga_ga.get_history()["max"])
        higa_results.append(higa_ga.get_history()["max"])
    
    return stnd_results, riga_results, eiga_results, higa_results, sp_set

In [7]:
def experiment_edge(graph_type, trials, generations):
    Pr = 0.05 # Probability to effect a change 
    n, k, p = 500, 4, 0.25 # WS params
    m = 3 # BA 
    degree = 2500 # LN 
    
    stnd_ga = GA() 
    riga_ga = GA()
    eiga_ga = GA()
    higa_ga = GA()
    
    # Run trials 
    stnd_results = []
    riga_results = []
    eiga_results = []
    higa_results = [] 
    sp_set = [] 
    
    for trial in range(trials):
        # Generate a new graph 
        if graph_type == "WS":
            G = weighted_watts_strogatz(n, k, p)
        elif graph_type == "BA":
            G = weighted_barabasi_albert(m, n)
        else:
            G = weighted_log_normal(n, degree)
        
        G_ = G.copy() # Copy to maintain original state G 
        src, dst = np.random.choice(n, size = 2, replace = False)
        # Initialise GA
        stnd_ga.initialise(G_, src, dst)
        riga_ga.initialise(G_, src, dst)
        eiga_ga.initialise(G_, src, dst)
        higa_ga.initialise(G_, src, dst)
        
        # Run generations 
        sp = []
        for generation in range(generations):            
            # Find true shortest path 
            path = nx.dijkstra_path(stnd_ga.G, src, dst, weight='weight')
            length = path_length(stnd_ga.G, path)
            sp.append(length ** (-1))
    
            # Run generation
            stnd_ga.run_generation()
            riga_ga.run_generation()
            eiga_ga.run_generation()
            higa_ga.run_generation() 
        
            # Update graph (avoid src, dst)
            dynamic_step_EDGE(G_, Pr)
            if not nx.has_path(G_, src, dst):
                G_ = G.copy()
            
            stnd_ga.set_graph(G_)
            riga_ga.set_graph(G_)
            eiga_ga.set_graph(G_)
            higa_ga.set_graph(G_) 
            
        # Get the results from the run  
        sp_set.append(sp)
        stnd_results.append(stnd_ga.get_history()["max"])
        riga_results.append(riga_ga.get_history()["max"])
        eiga_results.append(eiga_ga.get_history()["max"])
        higa_results.append(higa_ga.get_history()["max"])
    
    return stnd_results, riga_results, eiga_results, higa_results, sp_set

In [8]:
def path_length(G, p):
        return sum(G[i][j]['weight'] for i, j in zip(p, p[1:]))

**Run Experiments**
The experiments run the different GA configurations for several trials, for a given graph type. The true SP fitness (target fitness) set is also provided. From this, the difference between the actual and target fitness can be calculated.

In [37]:
# Number of trials to run set as 20... the higher the more accurate the result 
trials = 10
generations = 60 

ws_stnd, ws_riga, ws_eiga, ws_higa, ws_sp = experiment_edge("WS", trials, generations)
ba_stnd, ba_riga, ba_eiga, ba_higa, ba_sp = experiment_edge("BA", trials, generations)
ln_stnd, ln_riga, ln_eiga, ln_higa, ln_sp = experiment_edge("LN", trials, generations)

In [67]:
# Calculate the BGA for edge 
# c = np.divide(a, b, out=np.zeros_like(a), where=b!=0)

# Accuracy for every trial, every generation 
# ws
edge_ws_stnd_acc = np.divide(ws_stnd, ws_sp, out = np.zeros_like(ws_stnd), where=ws_stnd!=0)
edge_ws_riga_acc = np.divide(ws_riga, ws_sp, out = np.zeros_like(ws_riga), where=ws_riga!=0)
edge_ws_eiga_acc = np.divide(ws_eiga, ws_sp, out = np.zeros_like(ws_eiga), where=ws_eiga!=0)
edge_ws_higa_acc = np.divide(ws_higa, ws_sp, out = np.zeros_like(ws_higa), where=ws_higa!=0)
# ba
edge_ba_stnd_acc = np.divide(ba_stnd, ba_sp, out = np.zeros_like(ba_stnd), where=ba_stnd!=0)
edge_ba_riga_acc = np.divide(ba_riga, ba_sp, out = np.zeros_like(ba_riga), where=ba_riga!=0)
edge_ba_eiga_acc = np.divide(ba_eiga, ba_sp, out = np.zeros_like(ba_eiga), where=ba_eiga!=0)
edge_ba_higa_acc = np.divide(ba_higa, ba_sp, out = np.zeros_like(ba_higa), where=ba_higa!=0)
# ln
edge_ln_stnd_acc = np.divide(ln_stnd, ln_sp, out = np.zeros_like(ln_stnd), where=ln_stnd!=0)
edge_ln_riga_acc = np.divide(ln_riga, ln_sp, out = np.zeros_like(ln_riga), where=ln_riga!=0)
edge_ln_eiga_acc = np.divide(ln_eiga, ln_sp, out = np.zeros_like(ln_eiga), where=ln_eiga!=0)
edge_ln_higa_acc = np.divide(ln_higa, ln_sp, out = np.zeros_like(ln_higa), where=ln_higa!=0)

# Then take average for each trial 
edge_ws_stnd_bga = np.mean(np.mean(edge_ws_stnd_acc, axis = 0))
edge_ws_riga_bga = np.mean(np.mean(edge_ws_riga_acc, axis = 0))
edge_ws_eiga_bga = np.mean(np.mean(edge_ws_eiga_acc, axis = 0))
edge_ws_higa_bga = np.mean(np.mean(edge_ws_higa_acc, axis = 0))

edge_ba_stnd_bga = np.mean(np.mean(edge_ba_stnd_acc, axis = 0))
edge_ba_riga_bga = np.mean(np.mean(edge_ba_riga_acc, axis = 0))
edge_ba_eiga_bga = np.mean(np.mean(edge_ba_eiga_acc, axis = 0))
edge_ba_higa_bga = np.mean(np.mean(edge_ba_higa_acc, axis = 0))

edge_ln_stnd_bga = np.mean(np.mean(edge_ln_stnd_acc, axis = 0))
edge_ln_riga_bga = np.mean(np.mean(edge_ln_riga_acc, axis = 0))
edge_ln_eiga_bga = np.mean(np.mean(edge_ln_eiga_acc, axis = 0))
edge_ln_higa_bga = np.mean(np.mean(edge_ln_higa_acc, axis = 0))

0.17498000731476984
0.12860413477226632


In [71]:
import pandas as pd 
# Edge set block 
edge_set_bga = [[edge_ws_stnd_bga, edge_ws_riga_bga, edge_ws_eiga_bga, edge_ws_higa_bga],
                [edge_ba_stnd_bga, edge_ba_riga_bga, edge_ba_eiga_bga, edge_ba_higa_bga],
                [edge_ln_stnd_bga, edge_ln_riga_bga, edge_ln_eiga_bga, edge_ln_higa_bga]]

print("Edge Set BGA")
pd.DataFrame(edge_set_bga, columns = ("Standrad", "RIGA", "EIGA", "HIGA"), index = ("WS", "BA", "LN"))

Edge Set BGA


,Standrad,RIGA,EIGA,HIGA
WS,0.240832,0.301248,0.312507,0.261252
BA,0.256440,0.245533,0.224812,0.286415
LN,0.174980,0.106760,0.124692,0.128604


In [72]:
ws_stnd, ws_riga, ws_eiga, ws_higa, ws_sp = experiment_weight("WS", trials, generations)
ba_stnd, ba_riga, ba_eiga, ba_higa, ba_sp = experiment_weight("BA", trials, generations)
ln_stnd, ln_riga, ln_eiga, ln_higa, ln_sp = experiment_weight("LN", trials, generations)

# Accuracy for every trial, every generation 
# ws
weight_ws_stnd_acc = np.divide(ws_stnd, ws_sp, out = np.zeros_like(ws_stnd), where=ws_stnd!=0)
weight_ws_riga_acc = np.divide(ws_riga, ws_sp, out = np.zeros_like(ws_riga), where=ws_riga!=0)
weight_ws_eiga_acc = np.divide(ws_eiga, ws_sp, out = np.zeros_like(ws_eiga), where=ws_eiga!=0)
weight_ws_higa_acc = np.divide(ws_higa, ws_sp, out = np.zeros_like(ws_higa), where=ws_higa!=0)
# ba
weight_ba_stnd_acc = np.divide(ba_stnd, ba_sp, out = np.zeros_like(ba_stnd), where=ba_stnd!=0)
weight_ba_riga_acc = np.divide(ba_riga, ba_sp, out = np.zeros_like(ba_riga), where=ba_riga!=0)
weight_ba_eiga_acc = np.divide(ba_eiga, ba_sp, out = np.zeros_like(ba_eiga), where=ba_eiga!=0)
weight_ba_higa_acc = np.divide(ba_higa, ba_sp, out = np.zeros_like(ba_higa), where=ba_higa!=0)
# ln
weight_ln_stnd_acc = np.divide(ln_stnd, ln_sp, out = np.zeros_like(ln_stnd), where=ln_stnd!=0)
weight_ln_riga_acc = np.divide(ln_riga, ln_sp, out = np.zeros_like(ln_riga), where=ln_riga!=0)
weight_ln_eiga_acc = np.divide(ln_eiga, ln_sp, out = np.zeros_like(ln_eiga), where=ln_eiga!=0)
weight_ln_higa_acc = np.divide(ln_higa, ln_sp, out = np.zeros_like(ln_higa), where=ln_higa!=0)

# Then take average for each trial 
weight_ws_stnd_bga = np.mean(np.mean(weight_ws_stnd_acc, axis = 0))
weight_ws_riga_bga = np.mean(np.mean(weight_ws_riga_acc, axis = 0))
weight_ws_eiga_bga = np.mean(np.mean(weight_ws_eiga_acc, axis = 0))
weight_ws_higa_bga = np.mean(np.mean(weight_ws_higa_acc, axis = 0))

weight_ba_stnd_bga = np.mean(np.mean(weight_ba_stnd_acc, axis = 0))
weight_ba_riga_bga = np.mean(np.mean(weight_ba_riga_acc, axis = 0))
weight_ba_eiga_bga = np.mean(np.mean(weight_ba_eiga_acc, axis = 0))
weight_ba_higa_bga = np.mean(np.mean(weight_ba_higa_acc, axis = 0))

weight_ln_stnd_bga = np.mean(np.mean(weight_ln_stnd_acc, axis = 0))
weight_ln_riga_bga = np.mean(np.mean(weight_ln_riga_acc, axis = 0))
weight_ln_eiga_bga = np.mean(np.mean(weight_ln_eiga_acc, axis = 0))
weight_ln_higa_bga = np.mean(np.mean(weight_ln_higa_acc, axis = 0))

In [73]:
# Edge set block 
weight_set_bga = [[weight_ws_stnd_bga, weight_ws_riga_bga, weight_ws_eiga_bga, weight_ws_higa_bga],
                  [weight_ba_stnd_bga, weight_ba_riga_bga, weight_ba_eiga_bga, weight_ba_higa_bga],
                  [weight_ln_stnd_bga, weight_ln_riga_bga, weight_ln_eiga_bga, weight_ln_higa_bga]]

print("Weight Set BGA")
pd.DataFrame(weight_set_bga, columns = ("Standrad", "RIGA", "EIGA", "HIGA"), index = ("WS", "BA", "LN"))

Weight Set BGA


,Standrad,RIGA,EIGA,HIGA
WS,0.766668,0.819361,0.748040,0.814541
BA,0.600757,0.603461,0.630320,0.623035
LN,0.472843,0.588114,0.574488,0.578455


In [78]:
ws_stnd, ws_riga, ws_eiga, ws_higa, ws_sp = experiment_node("WS", trials, generations)
ba_stnd, ba_riga, ba_eiga, ba_higa, ba_sp = experiment_node("BA", trials, generations)
ln_stnd, ln_riga, ln_eiga, ln_higa, ln_sp = experiment_node("LN", trials, generations)

# Accuracy for every trial, every generation 
# ws
node_ws_stnd_acc = np.divide(ws_stnd, ws_sp, out = np.zeros_like(ws_stnd), where=ws_stnd!=0)
node_ws_riga_acc = np.divide(ws_riga, ws_sp, out = np.zeros_like(ws_riga), where=ws_riga!=0)
node_ws_eiga_acc = np.divide(ws_eiga, ws_sp, out = np.zeros_like(ws_eiga), where=ws_eiga!=0)
node_ws_higa_acc = np.divide(ws_higa, ws_sp, out = np.zeros_like(ws_higa), where=ws_higa!=0)
# ba
node_ba_stnd_acc = np.divide(ba_stnd, ba_sp, out = np.zeros_like(ba_stnd), where=ba_stnd!=0)
node_ba_riga_acc = np.divide(ba_riga, ba_sp, out = np.zeros_like(ba_riga), where=ba_riga!=0)
node_ba_eiga_acc = np.divide(ba_eiga, ba_sp, out = np.zeros_like(ba_eiga), where=ba_eiga!=0)
node_ba_higa_acc = np.divide(ba_higa, ba_sp, out = np.zeros_like(ba_higa), where=ba_higa!=0)
# ln
node_ln_stnd_acc = np.divide(ln_stnd, ln_sp, out = np.zeros_like(ln_stnd), where=ln_stnd!=0)
node_ln_riga_acc = np.divide(ln_riga, ln_sp, out = np.zeros_like(ln_riga), where=ln_riga!=0)
node_ln_eiga_acc = np.divide(ln_eiga, ln_sp, out = np.zeros_like(ln_eiga), where=ln_eiga!=0)
node_ln_higa_acc = np.divide(ln_higa, ln_sp, out = np.zeros_like(ln_higa), where=ln_higa!=0)

# Then take average for each trial 
node_ws_stnd_bga = np.mean(np.mean(node_ws_stnd_acc, axis = 0))
node_ws_riga_bga = np.mean(np.mean(node_ws_riga_acc, axis = 0))
node_ws_eiga_bga = np.mean(np.mean(node_ws_eiga_acc, axis = 0))
node_ws_higa_bga = np.mean(np.mean(node_ws_higa_acc, axis = 0))

node_ba_stnd_bga = np.mean(np.mean(node_ba_stnd_acc, axis = 0))
node_ba_riga_bga = np.mean(np.mean(node_ba_riga_acc, axis = 0))
node_ba_eiga_bga = np.mean(np.mean(node_ba_eiga_acc, axis = 0))
node_ba_higa_bga = np.mean(np.mean(node_ba_higa_acc, axis = 0))

node_ln_stnd_bga = np.mean(np.mean(node_ln_stnd_acc, axis = 0))
node_ln_riga_bga = np.mean(np.mean(node_ln_riga_acc, axis = 0))
node_ln_eiga_bga = np.mean(np.mean(node_ln_eiga_acc, axis = 0))
node_ln_higa_bga = np.mean(np.mean(node_ln_higa_acc, axis = 0))

In [81]:
# Node set block 
node_set_bga = [[node_ws_stnd_bga, node_ws_riga_bga, node_ws_eiga_bga, node_ws_higa_bga],
                  [node_ba_stnd_bga, node_ba_riga_bga, node_ba_eiga_bga, node_ba_higa_bga],
                  [node_ln_stnd_bga, node_ln_riga_bga, node_ln_eiga_bga, node_ln_higa_bga]]

print("Node Set BGA")
pd.DataFrame(node_set_bga, columns = ("Standrad", "RIGA", "EIGA", "HIGA"), index = ("WS", "BA", "LN"))

Node Set BGA


,Standrad,RIGA,EIGA,HIGA
WS,0.423773,0.424486,0.428482,0.390456
BA,0.331057,0.339533,0.375910,0.426892
LN,0.257281,0.273855,0.251358,0.185078


**How many generations achieve the target fitness?**

In [83]:
# For each row, count the number of elements that have value 1 
# Average number of times it 'hits the ceiling' per 60 generations 
edge_ws_stnd_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ws_stnd_acc])
edge_ws_riga_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ws_riga_acc])
edge_ws_eiga_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ws_eiga_acc])
edge_ws_higa_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ws_higa_acc])

edge_ba_stnd_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ba_stnd_acc])
edge_ba_riga_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ba_riga_acc])
edge_ba_eiga_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ba_eiga_acc])
edge_ba_higa_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ba_higa_acc])

edge_ln_stnd_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ln_stnd_acc])
edge_ln_riga_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ln_riga_acc])
edge_ln_eiga_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ln_eiga_acc])
edge_ln_higa_ceiling = np.mean([np.count_nonzero(n == 1) for n in edge_ln_higa_acc])

# Then get the average 
edge_set_ceiling = [[edge_ws_stnd_ceiling, edge_ws_riga_ceiling, edge_ws_eiga_ceiling, edge_ws_higa_ceiling],
 [edge_ba_stnd_ceiling, edge_ba_riga_ceiling, edge_ba_eiga_ceiling, edge_ba_higa_ceiling],
 [edge_ln_stnd_ceiling, edge_ln_riga_ceiling, edge_ln_eiga_ceiling, edge_ln_higa_ceiling]]

edge_set_ceiling_pd = pd.DataFrame(edge_set_ceiling, columns = ("Standard", "RIGA", "EIGA", "HIGA"), 
                                   index = ("WS", "BS", "LN"))

In [92]:
print("Edge Set Mean Generations at Target Fitness (/60 Generations)")
edge_set_ceiling_pd

Edge Set Mean Generations at Target Fitness (/60 Generations)


,Standard,RIGA,EIGA,HIGA
WS,3.1,4.2,4.8,2.9
BS,0.9,1.5,2.1,4.5
LN,0.7,0.0,0.1,0.1


In [87]:
weight_ws_stnd_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ws_stnd_acc])
weight_ws_riga_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ws_riga_acc])
weight_ws_eiga_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ws_eiga_acc])
weight_ws_higa_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ws_higa_acc])

weight_ba_stnd_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ba_stnd_acc])
weight_ba_riga_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ba_riga_acc])
weight_ba_eiga_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ba_eiga_acc])
weight_ba_higa_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ba_higa_acc])

weight_ln_stnd_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ln_stnd_acc])
weight_ln_riga_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ln_riga_acc])
weight_ln_eiga_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ln_eiga_acc])
weight_ln_higa_ceiling = np.mean([np.count_nonzero(n == 1) for n in weight_ln_higa_acc])

weight_set_ceiling = [[weight_ws_stnd_ceiling, weight_ws_riga_ceiling, weight_ws_eiga_ceiling, weight_ws_higa_ceiling],
 [weight_ba_stnd_ceiling, weight_ba_riga_ceiling, weight_ba_eiga_ceiling, weight_ba_higa_ceiling],
 [weight_ln_stnd_ceiling, weight_ln_riga_ceiling, weight_ln_eiga_ceiling, weight_ln_higa_ceiling]]

weight_set_ceiling_pd = pd.DataFrame(weight_set_ceiling, columns = ("Standard", "RIGA", "EIGA", "HIGA"), 
             index = ("WS", "BS", "LN"))

In [93]:
print("Weight Set Mean Generations at Target Fitness (/60 Generations)")
weight_set_ceiling_pd

Weight Set Mean Generations at Target Fitness (/60 Generations)


,Standard,RIGA,EIGA,HIGA
WS,8.1,9.6,6.8,11.2
BS,4.1,3.0,4.5,3.5
LN,9.0,12.1,11.8,11.7


In [90]:
node_ws_stnd_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ws_stnd_acc])
node_ws_riga_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ws_riga_acc])
node_ws_eiga_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ws_eiga_acc])
node_ws_higa_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ws_higa_acc])

node_ba_stnd_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ba_stnd_acc])
node_ba_riga_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ba_riga_acc])
node_ba_eiga_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ba_eiga_acc])
node_ba_higa_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ba_higa_acc])

node_ln_stnd_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ln_stnd_acc])
node_ln_riga_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ln_riga_acc])
node_ln_eiga_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ln_eiga_acc])
node_ln_higa_ceiling = np.mean([np.count_nonzero(n == 1) for n in node_ln_higa_acc])

node_set_ceiling = [[node_ws_stnd_ceiling, node_ws_riga_ceiling, node_ws_eiga_ceiling, node_ws_higa_ceiling],
 [node_ba_stnd_ceiling, node_ba_riga_ceiling, node_ba_eiga_ceiling, node_ba_higa_ceiling],
 [node_ln_stnd_ceiling, node_ln_riga_ceiling, node_ln_eiga_ceiling, node_ln_higa_ceiling]]

node_set_ceiling_pd = pd.DataFrame(node_set_ceiling, columns = ("Standard", "RIGA", "EIGA", "HIGA"), 
             index = ("WS", "BS", "LN"))

In [94]:
print("Node Set Mean Generations at Target Fitness (/60 Generations)")
node_set_ceiling_pd

Node Set Mean Generations at Target Fitness (/60 Generations)


,Standard,RIGA,EIGA,HIGA
WS,5.2,6.0,5.7,5.8
BS,8.8,8.5,11.9,12.8
LN,2.2,3.0,3.8,0.1
